Example response from getRepoContent where path is a file:

```json
{
  "name": "README.md",
  "path": "README.md",
  "sha": "eb997a5d18ce1c9ae0ad480dfc93d73b51a5b570",
  "size": 4853,
  "url": "https://api.github.com/repos/microsoft/vscode-docs/contents/README.md?ref=main",
  "html_url": "https://github.com/microsoft/vscode-docs/blob/main/README.md",
  "git_url": "https://api.github.com/repos/microsoft/vscode-docs/git/blobs/eb997a5d18ce1c9ae0ad480dfc93d73b51a5b570",
  "download_url": "https://raw.githubusercontent.com/microsoft/vscode-docs/main/README.md",
  "type": "file",
  "content": "PHAgYWxpZ249ImNlbnRlciI+CiAgPGltZyBhbHQ9InZzY29kZSBsb2dvIiBz\ncmM9ImltYWdlcy9sb2dvLXN0YWJsZS5wbmciIHdpZHRoPSIxMc3VhbCBT\ndHVkaW8gQ29kZSBkb2N1bWVudGF0aW9uIEdpdEh1YiByZXBvc2l0b3J5LCB3\naGljaCBjb250YWlucyB0aGUgY29udGVudCBmb3IgdGhlIFtWaXN1YWwgU3R1\nZGlvIENvZGUgZG9jdW1lbnRhdGlvbl0oaHR0cHM6Ly9jb2RlLnZpc3VhbHN0\ndWRpby5jb20vZG9jcykuCgpUb3BpY3Mgc3VibWl0dGVkIGhlcmUgd2lsbCBi\nZSBwdWJsaXNoZWQgdG8gdGhlIFtWaXN1YWwgU3R1ZGlvIENvZGVdKGh0dHBz\nOi8vY29kZS52aXN1YWxzdHVkaW8uY29tKSBwb3J0YWwuCgpJZiB5b3UgYXJl\nIGxvb2tpbmcgZm9yIHRoZSBWUyBDb2RlIHByb2R1Y3QgR2l0SHViIHJlcG9z\naXRvcnksIHlvdSBjYW4gZmluZCBpdCBbaGVyZV0oaHR0cHM6Ly9naXRodWIu\nY29tL21pY3Jvc29mdC92c2NvZGUpLgoKPioqTm90ZSoqOiBUaGUgdnNjb2Rl\nLWRvY3MgcmVwb3NpdG9yeSB1c2VzIFtHaXQgTEZTXShodHRwczovL2dpdC1s\nZnMuZ2l0aHViLmNvbS8pIChMYXJnZSBGaWxlIFN0b3JhZ2UpIGZvciBzdG9y\naW5nIGJpbmFyeSBmaWxlcyBzdWNoIGFzIGltYWdlcyBhbmQgYC5naWZgcy4g\nSWYgeW91IGFyZSBjb250cmlidXRpbmcgb3IgdXBkYXRpbmcgaW1hZ2VzLCBw\nbGVhc2UgZW5hYmxlIEdpdCBMRlMgcGVyIHRoZSBpbnN0cnVjdGlvbnMgaW4g\ndGhlIFtDb250cmlidXRpbmddKCNjbG9uaW5nKSBzZWN0aW9uIGJlbG93LgoK\nIyMgSW5kZXgKCi0gW0luZGV4XSgjaW5kZXgpCi0gW1Zpc3VhbCBTdHVkaW8g...",
  "encoding": "base64",
  "_links": {
    "self": "https://api.github.com/repos/microsoft/vscode-docs/contents/README.md?ref=main",
    "git": "https://api.github.com/repos/microsoft/vscode-docs/git/blobs/eb997a5d18ce1c9ae0ad480dfc93d73b51a5b570",
    "html": "https://github.com/microsoft/vscode-docs/blob/main/README.md"
  }
}
```


In [4]:
import { Octokit, App } from "https://esm.sh/octokit?dts";
import { fetch } from "https://esm.sh/node-fetch?dts";
import { v5 as uuidv5 } from "https://esm.sh/uuid?dts";
import { default as atob } from "https://esm.sh/atob?dts";

// import { atob } from "https://deno.land/std/encoding/base64.ts";

// token $GH_TOKEN
const octokit = new Octokit({
    auth: Deno.env.get("GH_TOKEN"),
    // request: {
    //     fetch: fetch,
    //   }
});

let owner = "microsoft";
let repo = "vscode";

const SLEEP_DURATION = 300;

async function getRepoContent(owner: string, repo: string, path?: string) {
    // trim path of '^/' and '/$'
    if (path) {
        path = path.replace(/^\/|\/$/g, "");
    }
    try {
        return await octokit.request("GET /repos/{owner}/{repo}/contents/{path}", {
            owner: owner,
            repo: repo,
            path: path,
        });
    } catch (e) {
        console.error(`Error fetching content for ${owner}/${repo}/${path}:`, e);
    }
}


// filenames to include when fetching dotfiles from a repository. 
// Do not include files that configure projects.
// - example of exclude: package.json, requirements.txt, .gitignore

const includeConfigFiles = [
    "uv.json",
    "ruff.json",
    "clippy.toml",
    "rustfmt.toml",
    "babel.config",
    "jest.config",
    "prettier.config",
    "webpack.config",
    "pylintrc",
    "flake8",
    "mypy.ini",
    "setup.cfg",
    "tox.ini",
    "pyproject.toml"
];
/**
 * Get dotfiles from a repository.
 *
 * @param {string} owner Owner of the repository
 * @param {string} repo Repository name
 * @returns {Promise<Array>} Array of objects where each object is a dotfile
 */
async function getRepoDotfileContents(owner: string, repo: string) {
    const contents = await getRepoContent(owner, repo);
    // filter to objects where type="file"
    // filter to objects where (
    //   name starts with "." or
    //   any of the includeConfigFiles is a substring of the name (case-insensitive)
    // )
    return contents.data.filter(
        (x: { type: string; name: string }) =>
            x.type == "file" && (x.name.startsWith(".") || includeConfigFiles.some((y) => x.name.toLowerCase().includes(y.toLowerCase())))
    );
}

async function getRepoContentFile(owner: string, repo: string, path: string) {
    const contents = await getRepoContent(owner, repo, path);
    // type needs to equal "file"
    if (contents.data.type != "file") {
        throw new Error("Not a file");
    }
    // get encoded content "content" and decode with "encoding"
    if (contents.data.encoding != "base64") {
        throw new Error("Not base64 encoded");
    }
    const content = atob(contents.data.content);
    return content;
}

/** 
load text file from $TEMPDIR/repos.list
split by line, trim each line
remove empty lines
unique lines
split by / to get owner and repo per line
*/

/**
 * Load a list of repositories from a specified file.
 *
 * @param {string} path Path to the file containing the list of repositories
 *
 * @returns {Array<Array<string>>} Array of arrays where each inner array is [owner, repo]
 */
function loadRepoList(path?: string) {
    // if not path, use $TEMPDIR/repos.list
    if (!path) {
        path = Deno.env.get("TEMPDIR") + "/repos.list";
    }
    let f = Deno.readTextFileSync(path);
    let repos = f
        .split("\n")
        .map((x) => x.trim())
        .filter((x) => x.length > 0)
        .filter((v, i, a) => a.indexOf(v) === i)
        .map((x) => x.split("/"));
    return repos;
}


function dumpDotfileContents(dotfileContents: any, dst: string = Deno.env.get("TEMPDIR") + "/repoDotfilesQuery.json") {
    let dotfileContentsString = JSON.stringify(dotfileContents, null, 2);
    // overwrite file if exists
    Deno.writeTextFileSync(dst, dotfileContentsString);
}


// load flat list of dotfiles from $TEMPDIR/repoDotfilesQueryFlat.json if exists
// if not exists, create empty list
function loadDotfileContentsFlat(path: string = Deno.env.get("TEMPDIR") + "/repoDotfilesQueryFlat.json") {
    try {
        let f = Deno.readTextFileSync
        let dotfileContents = JSON.parse(f);
        return dotfileContents;
    } catch (e) {
        console.error(`Error loading dotfile contents from ${path}:`, e);
        return [];
    }
};

const dotfileIndex = loadDotfileContentsFlat();

/**
 * Get boolean if a dotfile is in the index. 
 *
 * @param owner 
 * @param repo 
 * @param name 
 * @returns {boolean} True if dotfile is in index, false otherwise
 */
function isInIndex(owner: string, repo: string, name: string) {
    return dotfileIndex.some((x: { owner: string; repo: string; name: string }) => x.owner == owner && x.repo == repo && x.name == name);
};

/**
 * Fetch dotfiles from a list of repositories.
 *
 * @param {Array<Array<string>>} reposList Array of arrays where each inner array is [owner, repo]
 * @param {number} sleepDuration Sleep duration between each fetch
 */
async function fetchDotfilesFromRepos(
    reposList: Array<Array<string>>,
    sleepDuration: number = 1000
) {
    const dotfileContent = [];
    const dotfileContentFlat = []

    for (const [owner, repo] of reposList) {
        try {
            let dotfiles = await getRepoDotfileContents(owner, repo);
            // console.debug(`Dotfiles for ${owner}/${repo}:`, dotfiles);
            /**
             * Object that is returned at end of this function
             */
            let repoDotfilesContent: {
                owner: string;
                repo: string;
                dotfiles: { name: string; content: string }[];
            } = {
                owner: owner,
                repo: repo,
                dotfiles: [],
            };
            for (const dotfile of dotfiles) {
                if (isInIndex(owner, repo, dotfile.name)) {
                    console.debug(`Skipping ${owner}/${repo}/${dotfile.name} as it is already in the index`);
                    continue;
                }
                try {
                    let content = await getRepoContentFile(owner, repo, dotfile.path);
                    repoDotfilesContent.dotfiles.push({ name: dotfile.name, content: content });

                    // add to dotfileContentFlat
                    dotfileContentFlat.push({ owner: owner, repo: repo, name: dotfile.name, content: content });
                    await new Promise((r) => setTimeout(r, sleepDuration));
                } catch (e) {
                    console.error(`Error fetching dotfile ${dotfile.name} for ${owner}/${repo}:`, e);
                }
            }
            dotfileContent.push(repoDotfilesContent);
            dumpDotfileContents(dotfileContent, Deno.env.get("TEMPDIR") + "/repoDotfilesQuery.json");
            dumpDotfileContents(dotfileContentFlat, Deno.env.get("TEMPDIR") + "/repoDotfilesQueryFlat.json");
            // console.debug(dotfilesContent);
        } catch (e) {
            console.error(`Error fetching dotfiles for ${owner}/${repo}:`, e);
        }

    }
    return dotfileContent;
}


let reposList = loadRepoList();
// only fetch from first 5 repos
let repoDotfileContents = await fetchDotfilesFromRepos(reposList, SLEEP_DURATION);


Error loading dotfile contents from /tmp/user/1000/repoDotfilesQueryFlat.json: SyntaxError: Unexpected token 'u', "function re"... is not valid JSON
    at JSON.parse (<anonymous>)
    at loadDotfileContentsFlat (<anonymous>:106:32)
    at <anonymous>:113:22
    at eventLoopTick (ext:core/01_core.js:177:7)
Error fetching content for AnthoPakPak/StackOverflowPowerUser?/undefined: HttpError: Not Found - https://docs.github.com/rest/repos/contents#get-repository-content
    at g (https://esm.sh/@octokit/request@9.2.2/denonext/request.mjs:2:2166)
    at eventLoopTick (ext:core/01_core.js:177:7)
    at async f (https://esm.sh/@octokit/plugin-retry@7.1.4/denonext/plugin-retry.mjs:2:716)
    at async N.doExecute (https://esm.sh/bottleneck@2.19.5/denonext/light.mjs:2:5696) {
  name: "HttpError",
  status: 404,
  request: {
    method: "GET",
    url: "https://api.github.com/repos/AnthoPakPak/StackOverflowPowerUser%3F/contents",
    headers: {
      accept: "application/vnd.github.v3+json",
   

In [2]:
console.log(Deno.env.get("TEMPDIR") + "/repoDotfilesQuery.json");
console.log(Deno.env.get("TEMPDIR") + "/repoDotfilesQueryFlat.json");

/tmp/user/1000/repoDotfilesQuery.json
/tmp/user/1000/repoDotfilesQueryFlat.json


In [37]:
repoDotfileContents

[
  {
    owner: "abraunegg",
    repo: "onedrive",
    dotfiles: [
      {
        name: ".gitignore",
        content: ".*\n" +
          "onedrive\n" +
          "onedrive.1\n" +
          "onedrive.o\n" +
          "version\n" +
          "Makefile\n" +
          "config.log\n" +
          "config.status\n" +
          "autom4te.cache/\n" +
          "contrib/pacman/PKGBUILD\n" +
          "contrib/spec/onedrive.spec\n" +
          "\n" +
          "# Ignore everything in the .github folder\n" +
          ".github/*\n" +
          "\n" +
          "# Allow actions/spelling in .github\n" +
          "!.github/actions/spelling/\n" +
          "\n" +
          "# Ensure spellcheck.yaml in .github is not ignored\n" +
          "!.github/actions/spelling/spellcheck.yaml"
      }
    ]
  },
  {
    owner: "astral-sh",
    repo: "ruff",
    dotfiles: [
      {
        name: ".editorconfig",
        content: "# Check http://editorconfig.org for more information\n" +
          "# This is th